# Variables

In [1]:
ga_input = '/grid/home/nbourgeois/data/test_proteins/pah/pah.pep.fas' #sequence pep.fas Input
tissue = 'Liver' # Tissue type for CoCoPuts
filename = 'bai_1'

# Imports


In [2]:
import os
cwd = os.getcwd()

## Import the code for codon opt
import sys
sys.path.insert(1, '/grid/home/nbourgeois/codonOpt')
# from run_pygad import *
from general_functions import *
from dyn_prog import *
from run_pygad import *


In [3]:
(keys, seqs) = readFasta(ga_input)
if len(seqs) == 1:
    aa_seq=str(seqs[0])

In [4]:
wt_loc = '/grid/home/nbourgeois/data/codon_jason/wt_pah.fa'
a,seq = readFasta(wt_loc)
wt_seq = str(seq[0])

## Test Max

In [5]:
optimizer = Optimizer(aa_seq, tissues='Liver')
optimized_seq = optimizer.optimize()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

## Test Ramp

In [6]:
optimizer = Optimizer(aa_seq, tissues='Liver', ramp=True)
optimized_seq = optimizer.optimize()

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

## Test OG Ramp

In [7]:
optimizer = Optimizer(aa_seq, tissues='Liver', og_ramp=True, og_seq=wt_seq)
optimized_seq = optimizer.optimize()

TypeError: __init__() got an unexpected keyword argument 'og_ramp'

## Test Multi Weight

In [ ]:
weights = pd.read_csv('test_data/weights.txt',sep='\t')
tissue_list = weights['Tissue'].values.tolist()
prefix_codon = 'AAG'

In [ ]:
tissue_list = tissue_list[1:4]

In [ ]:
optimizer = Optimizer(aa_seq, tissues=tissue_list, prefix_codon=prefix_codon)
optimized_seq = optimizer.optimize()
optimizer = Optimizer(aa_seq, tissues=tissue_list, ntissues='Liver', prefix_codon=prefix_codon)
optimized_seq = optimizer.optimize()


# Test CAI

In [ ]:
cns_aa_seq = 'KLLLVPRELSGSSPVLEETHPAHQQGASRPGPRDAQAHPGRPRAVPTQCDVPPNSRFDCAPDKAITQEQCEARGCCYIPAKQGLQGAQMGQPWCFFPPSYPSYKLENLSSSEMGYTATLTRTTPTFFPKDILTLRLDVMMETENRLHFTIKDPANRRYEVPLETPHVHSRAPSPLYSVEFSEEPFGVIVRRQLDGRVLLNTTVAPLFFADQFLQLSTSLPSQYITGLAEHLSPLMLSTSWTRITLWNRDLAPTPGANLYGSHPFYLALEDGGSAHGVFLLNSNAMDVVLQPSPALSWRSTGGILDVYIFLGPEPKSVVQQYLDVVGYPFMPPYWGLGFHLCRWGYSSTAITRQVVENMTRAHFPLDVQWNDLDYMDSRRDFTFNKDGFRDFPAMVQELHQGGRRYMMIVDPAISSSGPAGSYRPYDEGLRRGVFITNETGQPLIGKVWPGSTAFPDFTNPTALAWWEDMVAEFHDQVPFDGMWIDMNEPSNFIRGSEDGCPNNELENPPYVPGVVGGTLQAATICASSHQFLSTHYNLHNLYGLTEAIASHRALVKARGTRPFVISRSTFAGHGRYAGHWTGDVWSSWEQLASSVPEILQFNLLGVPLVGADVCGFLGNTSEELCVRWTQLGAFYPFMRNHNSLLSLPQEPYSFSEPAQQAMRKALTLRYALLPHLYTLFHQAHVAGETVARPLFLEFPKDSSTWTVDHQLLWGEALLITPVLQAGKAEVTGYFPLGTWYDLQTVPVEALGSLPPPPAAPREPAIHSEGQWVTLPAPLDTINVHLRAGYIIPLQGPGLTTTESRQQPMALAVALTKGGEARGELFWDDGESLEVLERGAYTQVIFLARNNTIVNELVRVTSEGAGLQLQKVTVLGVATAPQQVLSNGVPVSNFTYSPDTKVLDICVSLLMGEQFLVSWC'

In [ ]:
optimizer = Optimizer(cns_aa_seq, tissues='Artery_Aorta', ntissues='Liver', prefix_codon=prefix_codon)
optimizer.method='CAI'
optimized_seq = optimizer.optimize()
cai_mets =  get_cai(optimized_seq,optimizer.cai_weight_dict['Liver']),get_cai(optimized_seq,optimizer.cai_weight_dict['Artery_Aorta'])
cai_diff = - get_cai(optimized_seq,optimizer.cai_weight_dict['Liver']) + get_cai(optimized_seq,optimizer.cai_weight_dict['Artery_Aorta'])

In [ ]:
cai_diff

In [ ]:
optimizer = Optimizer(cns_aa_seq, tissues='Artery_Aorta', ntissues='Liver', prefix_codon=prefix_codon)
optimized_seq = optimizer.optimize()
bai_mets =  (get_bai(optimized_seq,optimizer.bai_weight_dict['Liver']),get_bai(optimized_seq,optimizer.bai_weight_dict['Artery_Aorta']))
bai_diff =  get_bai(optimized_seq,optimizer.bai_weight_dict['Artery_Aorta'])- get_bai(optimized_seq,optimizer.bai_weight_dict['Liver'])

In [ ]:
bai_diff, bai_mets

In [ ]:
optimizer = Optimizer(cns_aa_seq, tissues='Liver', ntissues='Artery_Aorta', prefix_codon=prefix_codon)
optimized_seq = optimizer.optimize()
get_bai(optimized_seq,optimizer.bai_weight_dict['Liver']),get_bai(optimized_seq,optimizer.bai_weight_dict['Artery_Aorta'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects  as go
import plotly
colors=['rgb(0,50,90)',
        'rgb(246,126,94)',
        'rgb(106,118,132)',
        'rgb(116,81,173)',
        'rgb(20,103,172)',
        'rgb(41,171,226)',
        'rgb(51,51,51)',
       'rgb(0,50,90)']

In [ ]:
df = pd.DataFrame()
df['variable'] = ['CAI','CAI','BAI','BAI']
df['value'] = cai_mets + bai_mets
df['Tissue'] = ['Liver','Heart']*2

In [ ]:
color_discrete_map={}
i = 0
for x in df['Tissue'].unique(): 
    color_discrete_map[x] = colors[i]
    i +=1



fig = px.bar(df, x='variable', y='value', color='Tissue',barmode="group",
    color_discrete_map=color_discrete_map, 
    labels={
                 "diff": "Heart and Liver Difference",
                 "value": "Value",
                 "variable" : 'Metric'
             },
    title='Differentially Optimized Vector',
                 width=550, height=500
    )
layout = go.Layout(
  margin=go.layout.Margin(
        t=70, #top margin
    )
)
# fig.update_traces(marker_size=7)
fig.update_layout(layout)
fig.update_layout(title_y=.9, title_x=.5)
fig.show()

In [ ]:

# plt.rcParams['figure.figsize'] = [5, 4]
# ax = sns.barplot(data=df,x='variable',y='metric',hue='Tissue',dodge=True)
# plt.title('Differentially Optimized Vector', size = 16)
# plt.ylabel('Metric Score', size = 13)
# plt.xlabel('Metric', size = 13)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
